In [57]:
# OPERATIONAL VERSION BASED ON PREPROCESS.IPYNB

In [58]:
# Libraries
import pytesseract                      # Optical Character Recognition
import cv2                              # Image Modification
import os                               # Directory handling
import pandas as pd                     # Excel sheets
import xlsxwriter                       # Modifying excel sheets
import matplotlib.pyplot as plt         # Displaying images and plots during debugging
import numpy as np                      # Handling arrays

# Tesseract Setup
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\Elena.Justo\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

In [59]:
# Function:    processImage
#    Input:    Path to image to process.
#   Output:    Preprocessed image ready for OCR.

def processImage(imagePath):

    # Get Image
    image = cv2.imread(imagePath)                           # Import image
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)     # Turn it into grayscale

    # Apply Contrast
    alpha = 1.5  
    beta = 10
    contrasted = cv2.convertScaleAbs(imageGray, alpha=alpha, beta=beta) 

    # Bilateral Filter
    filtered = cv2.bilateralFilter(contrasted,9,75,75)

    # Segment the image by appling thresholds
    thresholded = cv2.adaptiveThreshold(filtered,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)

    # Remove background
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,20))   # Make kernel
    morph = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)   # Apply morphology
    
    result = cv2.bitwise_and(contrasted, contrasted, mask=morph)     # Apply mask to image

    return result

In [60]:
# Function:    setupColumns
#    Input:    Name of dataframe to modify.
#   Output:    Modifies input dataframe when called.

def setupColumns(dataframe):
    # Create new column for images
    dataframe.insert(1, "Processed Image", None, False)

    # Create new column for results
    dataframe.insert(2, "Raw OCR", None, False)

    # Create new column for suggested serial number
    dataframe.insert(3, "Suggested Serial Number", None, False)

    # Create new column for suggested brand
    dataframe.insert(4, "Suggested Brand", None, False)

    # Create new column for suggested brand
    dataframe.insert(5, "Suggested Model", None, False)

In [61]:
# Function:    setupDataframe
#    Input:    Path to site photo directory.
#   Output:    Returns dataframe ready to be converted into Excel sheet.

def setupDataframe(siteDir):
    # Target folder
    dir = siteDir

    # Create dataframe file to serve as UI when converted into excel file
    df = pd.DataFrame(data=[])

    # Getting file paths
    filenames = []
    for item in os.listdir(dir):
        filenames.append(item)

    filepaths = []
    for item in os.listdir(dir):
        filepaths.append(dir + "\\" + item)
        
    # Debug
    print(filenames)
    print(filepaths)

    # Save file names into UI
    df["File Name"] = filenames

    # Columns
    setupColumns(df)

    return df, filenames, filepaths

In [62]:
# Function:    setupExcel
#    Input:    Filename for excel, Name of dataframe to convert.
#   Output:    Returns workbook and worksheet object

def setupExcel(filename, dataframe):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object.
    dataframe.to_excel(writer, sheet_name='Sheet1')

    # Get the xlsxwriter workbook and worksheet objects.
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    # Setting column widths
    worksheet.set_column(first_col=0, last_col=0, width=10)      # Index column width
    worksheet.set_column(first_col=1, last_col=1, width=15)      # Filename column width
    worksheet.set_column(first_col=2, last_col=8, width=25)      # Image columns width

    return writer, workbook, worksheet

In [69]:
def iterateImages(sitedir, filenames, filepaths, excelfile, dataframe):
    # OCR Results
    results  = []

    # Create temp folder to store pre-processing of images
    try:
        os.mkdir("temp")
    except FileExistsError:
        print("File already exists")

    # Clean temp folder
    for files in os.listdir("temp"):
        os.remove("temp\\"+files)

    # Setup excel file
    writer, workbook, worksheet = setupExcel(excelfile, dataframe)

    for paths in filepaths:
        image = cv2.imread(paths)

        processed = processImage(sitedir + "\\" + filenames[filepaths.index(paths)])

        # Save processed image
        cv2.imwrite("temp\\"+ "processed_" +filenames[filepaths.index(paths)], processed)

        # Insert the processed image
        worksheet.set_row(row=filepaths.index(paths)+1, height=200)                                             
        worksheet.embed_image(filepaths.index(paths)+1, 2, "temp\\"+ "processed_" +filenames[filepaths.index(paths)])

        # OCR
        ocr_result = pytesseract.image_to_string("temp\\"+ "processed_" +filenames[filepaths.index(paths)])
        ocr_result = ocr_result.split("\n")

        if ocr_result == " " or ocr_result == "":
            worksheet.write(filepaths.index(paths)+1, 3, "unreadable")
            continue
        else:
            results.append(ocr_result)
            worksheet.write(filepaths.index(paths)+1, 3, str(ocr_result))

            print(filenames[filepaths.index(paths)])
            #print(ocr_result)

        # Filter OCR results for strings of interest
        for word in ocr_result:

            # Hussmann Serial Numbers
            if "SERIAL" in word:
                result = word[word.index("SERIAL"):17]
                worksheet.write(filepaths.index(paths)+1, 4, result)

            # RDC Brands
            if "HU" in word:
                worksheet.write(filepaths.index(paths)+1, 5, "HUSSMANN")

            if "ARN" in word:
                worksheet.write(filepaths.index(paths)+1, 5, "ARNEG")

            if "tral" in word:
                worksheet.write(filepaths.index(paths)+1, 5, "AUSTRAL")
            
            # Model Number Test
            if "KG" in word:
                result = word[word.index("KG"):]
                worksheet.write(filepaths.index(paths)+1, 6, result)

            if "LIS" in word:
                result = word[word.index("LIS"):]
                worksheet.write(filepaths.index(paths)+1, 6, result)
            
            if "Lis" in word:
                result = word[word.index("Lis"):]
                worksheet.write(filepaths.index(paths)+1, 6, result)

            if "KM" in word:
                result = word[word.index("KM"):]
                worksheet.write(filepaths.index(paths)+1, 6, result)

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()

In [64]:
df, names, paths = setupDataframe(r"sites\cessnock")
iterateImages(r"sites\cessnock", names, paths, "cessnock.xlsx", df)

['img (1).jpg', 'img (10).jpg', 'img (11).jpg', 'img (12).jpg', 'img (13).jpg', 'img (14).jpg', 'img (15).jpg', 'img (2).jpg', 'img (3).jpg', 'img (4).jpg', 'img (5).jpg', 'img (6).jpg', 'img (7).jpg', 'img (8).jpg', 'img (9).jpg']
['sites\\cessnock\\img (1).jpg', 'sites\\cessnock\\img (10).jpg', 'sites\\cessnock\\img (11).jpg', 'sites\\cessnock\\img (12).jpg', 'sites\\cessnock\\img (13).jpg', 'sites\\cessnock\\img (14).jpg', 'sites\\cessnock\\img (15).jpg', 'sites\\cessnock\\img (2).jpg', 'sites\\cessnock\\img (3).jpg', 'sites\\cessnock\\img (4).jpg', 'sites\\cessnock\\img (5).jpg', 'sites\\cessnock\\img (6).jpg', 'sites\\cessnock\\img (7).jpg', 'sites\\cessnock\\img (8).jpg', 'sites\\cessnock\\img (9).jpg']
File already exists
img (1).jpg
img (10).jpg
img (11).jpg
img (12).jpg
img (13).jpg
img (14).jpg
img (15).jpg
img (2).jpg
img (3).jpg
img (4).jpg
img (5).jpg
img (6).jpg
img (7).jpg
img (8).jpg
img (9).jpg


In [65]:
df, names, paths = setupDataframe(r"sites\cooma")
iterateImages(r"sites\cooma", names, paths, "cooma.xlsx", df)

['IMG_6419.JPG', 'IMG_6420.JPG', 'IMG_6421.JPG', 'IMG_6422.JPG', 'IMG_6423.JPG', 'IMG_6441.JPG', 'IMG_6442.JPG', 'IMG_6444.JPG', 'IMG_6445.JPG', 'IMG_6448.JPG', 'IMG_6449.JPG', 'IMG_6451.JPG', 'IMG_6452.JPG']
['sites\\cooma\\IMG_6419.JPG', 'sites\\cooma\\IMG_6420.JPG', 'sites\\cooma\\IMG_6421.JPG', 'sites\\cooma\\IMG_6422.JPG', 'sites\\cooma\\IMG_6423.JPG', 'sites\\cooma\\IMG_6441.JPG', 'sites\\cooma\\IMG_6442.JPG', 'sites\\cooma\\IMG_6444.JPG', 'sites\\cooma\\IMG_6445.JPG', 'sites\\cooma\\IMG_6448.JPG', 'sites\\cooma\\IMG_6449.JPG', 'sites\\cooma\\IMG_6451.JPG', 'sites\\cooma\\IMG_6452.JPG']
File already exists
IMG_6419.JPG
IMG_6420.JPG
IMG_6421.JPG
IMG_6422.JPG
IMG_6423.JPG
IMG_6441.JPG
IMG_6442.JPG
IMG_6444.JPG
IMG_6445.JPG
IMG_6448.JPG
IMG_6449.JPG
IMG_6451.JPG
IMG_6452.JPG


In [70]:
df, names, paths = setupDataframe(r"sites\rosemeadow")
iterateImages(r"sites\rosemeadow", names, paths, "rosemeadow.xlsx", df)

['IMG_7240.JPG', 'IMG_7244.JPG', 'IMG_7246.JPG', 'IMG_7248.JPG', 'IMG_7250.JPG', 'IMG_7264.JPG', 'IMG_7266.JPG', 'IMG_7268.JPG', 'IMG_7270.JPG', 'IMG_7272.JPG', 'IMG_7274.JPG', 'IMG_7276.JPG', 'IMG_7278.JPG', 'IMG_7280.JPG', 'IMG_7282.JPG', 'IMG_7284.JPG', 'IMG_7286.JPG', 'IMG_7288.JPG', 'IMG_7290.JPG', 'IMG_7292.JPG', 'IMG_7294.JPG', 'IMG_7296.JPG', 'IMG_7298.JPG', 'IMG_7300.JPG', 'IMG_7302.JPG', 'IMG_7304.JPG', 'IMG_7306.JPG', 'IMG_7308.JPG', 'IMG_7313.JPG', 'IMG_7315.JPG', 'IMG_7317.JPG', 'IMG_7319.JPG', 'IMG_7321.JPG', 'IMG_7323.JPG', 'IMG_7325.JPG', 'IMG_7327.JPG', 'IMG_7329.JPG', 'IMG_7331.JPG', 'IMG_7333.JPG', 'IMG_7335.JPG', 'IMG_7343.JPG', 'IMG_7345.JPG', 'IMG_7347.JPG', 'IMG_7349.JPG', 'IMG_7350.JPG', 'IMG_7352.JPG', 'IMG_7354.JPG', 'IMG_7356.JPG', 'IMG_7358.JPG', 'IMG_7360.JPG', 'IMG_7362.JPG', 'IMG_7364.JPG', 'IMG_7366.JPG', 'IMG_7368.JPG', 'IMG_7370.JPG', 'IMG_7372.JPG', 'IMG_7374.JPG', 'IMG_7376.jpg', 'IMG_7378.JPG', 'IMG_7382.jpg', 'IMG_7384.jpg', 'IMG_7386.jpg', 'IMG_73